In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
dataset="rideshare_kaggle.csv"
dataset = pd.read_csv(dataset,error_bad_lines=False,engine='python')
df2=pd.DataFrame(dataset)

In [ ]:
df.head(n=1)

,timestamp,hour,day,month,source,destination,cab_type,name,price,distance,surge_multiplier,latitude,longitude,temperature,apparentTemperature,short_summary,long_summary,precipIntensity,precipProbability,humidity,windSpeed,windGust,windGustTime,visibility,temperatureHigh,temperatureHighTime,temperatureLow,temperatureLowTime,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureLowTime,icon,dewPoint,pressure,windBearing,cloudCover,uvIndex,visibility.1,ozone,sunriseTime,sunsetTime,moonPhase,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,1.544953e+09,9,16,12,Haymarket Square,North Station,Lyft,Shared,5.0,0.44,1.0,42.2148,-71.033,42.34,37.12,Mostly Cloudy,Rain throughout the day.,0.0,0.0,0.68,8.66,9.17,1545015600,10.0,43.68,1544968800,34.19,1545048000,1544968800,27.39,1545044400,partly-cloudy-night,32.7,1021.98,57,0.72,0,10.0,303.8,1544962084,1544994864,0.3,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000


**PreProcessing**

In [ ]:
#df.head(n=1)
df=df2.drop(axis=1,columns=['id','datetime','timezone','apparentTemperatureHigh','product_id'])
df=df.fillna(0)
#rearrangine columns to target prive
df = df[['timestamp', 'hour', 'day', 'month', 'source', 'destination', 'cab_type', 'name', 'distance', 'surge_multiplier', 'latitude', 'longitude', 'temperature', 'apparentTemperature', 'short_summary', 'long_summary', 'precipIntensity', 'precipProbability', 'humidity', 'windSpeed', 'windGust', 'windGustTime', 'visibility', 'temperatureHigh', 'temperatureHighTime', 'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHighTime', 'apparentTemperatureLow', 'apparentTemperatureLowTime', 'icon', 'dewPoint', 'pressure', 'windBearing', 'cloudCover', 'uvIndex', 'visibility.1', 'ozone', 'sunriseTime', 'sunsetTime', 'moonPhase', 'precipIntensityMax', 'uvIndexTime', 'temperatureMin', 'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin', 'apparentTemperatureMinTime', 'apparentTemperatureMax', 'apparentTemperatureMaxTime','price']]
cols=list(df.columns.values)
print(cols)

['timestamp', 'hour', 'day', 'month', 'source', 'destination', 'cab_type', 'name', 'distance', 'surge_multiplier', 'latitude', 'longitude', 'temperature', 'apparentTemperature', 'short_summary', 'long_summary', 'precipIntensity', 'precipProbability', 'humidity', 'windSpeed', 'windGust', 'windGustTime', 'visibility', 'temperatureHigh', 'temperatureHighTime', 'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHighTime', 'apparentTemperatureLow', 'apparentTemperatureLowTime', 'icon', 'dewPoint', 'pressure', 'windBearing', 'cloudCover', 'uvIndex', 'visibility.1', 'ozone', 'sunriseTime', 'sunsetTime', 'moonPhase', 'precipIntensityMax', 'uvIndexTime', 'temperatureMin', 'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin', 'apparentTemperatureMinTime', 'apparentTemperatureMax', 'apparentTemperatureMaxTime', 'price']


In [ ]:
#label encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['source'] = encoder.fit_transform(df['source'].values)
df['destination'] = encoder.fit_transform(df['destination'].values)
df['cab_type'] = encoder.fit_transform(df['cab_type'].values)
df['name'] = encoder.fit_transform(df['name'].values)
df['short_summary'] = encoder.fit_transform(df['short_summary'].values)
df['long_summary'] = encoder.fit_transform(df['long_summary'].values)
df['icon']=df['icon'].astype(str)
df['icon'] = encoder.fit_transform(df['icon'].values)
#droping rows for unseen data
x_unseen=df.iloc[6396:6397,:-1]
df=df.drop(axis=0,index=[696])
df=df.sample(n=100)
X= df.iloc[:,:-1].values #features
y = df.iloc[:,-1].values #labels

In [ ]:
#holdout method
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# **Linear Regression**

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
loo = LeaveOneOut()
loo.get_n_splits(X)
y_true, y_predL,x_true = list(), list(),list()
for train_index, test_index in loo.split(X):
  x_train, x_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  model = LinearRegression()
  model = LinearRegression().fit(x_train, y_train)
  yhat = model.predict(x_test)
  x_true.append(x_test[0])
  y_true.append(y_test[0])
  y_predL.append(yhat[0])

In [ ]:
#prediction on unseen
y_gen = model.predict(x_unseen)
print(y_gen)

[7.68850141]


Performance of Linear regression

In [ ]:
#performance r2 and mean absolute error
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
#print("model.coef_:{}".format(model.coef_))
#print("model.intercept_:{}".format(model.intercept_))
print("training score {:.2f}",format(model.score(x_train, y_train)))
print("testinging score {:.2f}",format(model.score(x_true, y_true)))
print(mean_absolute_error(y_true, y_predL))

training score {:.2f} 0.7419450017726168
testinging score {:.2f} 0.7375270018991805
7.574424972092384


# **KNN Regression**

In [ ]:
from sklearn.model_selection import GridSearchCV
import sklearn.neighbors as neighbors
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
loo = LeaveOneOut()
loo.get_n_splits(X)
print(loo)
y_trueK, y_pred_knn,x_trueK = list(), list(),list()
for train_index, test_index in loo.split(X):
  x_train, x_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  params = {'n_neighbors':[2,3,4,5,6,7,8,9]}
  knn = neighbors.KNeighborsRegressor()
  model_knn = GridSearchCV(knn, params, cv=5)
  model_knn.fit(x_train,y_train)
  model_knn.best_params_
  yhat = model_knn.predict(x_test)
	# store
  x_trueK.append(x_test[0])
  y_trueK.append(y_test[0])
  y_pred_knn.append(yhat[0])

LeaveOneOut()


Performance of knn Regressor

In [ ]:
#performance r2 and mean absolute error
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
#print("model.coef_:{}".format(model.coef_))
#print("model.intercept_:{}".format(model.intercept_))
print("training score {:.2f}",format(model_knn.score(x_train, y_train)))
print("testinging score {:.2f}",format(model_knn.score(x_true, y_true)))
print(mean_absolute_error(y_true, y_pred_knn))

training score {:.2f} 0.21113729514122437
testinging score {:.2f} 0.21156522148193302
7.184047619047617


# **Ensemble Learning**

In [25]:
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
loo = LeaveOneOut()
loo.get_n_splits(X)
print(loo)
y_true, y_pred_esm,x_true = list(), list(),list()
for train_index, test_index in loo.split(X):
  x_train, x_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  scaler = StandardScaler()
  scaler.fit(x_train)
  x_train = scaler.transform(x_train)
  x_test = scaler.transform(x_test)
  models = [('model',LinearRegression()),('model_knn',neighbors.KNeighborsRegressor())]
  clf= StackingRegressor(estimators=models, final_estimator=LinearRegression())
  clf.fit(x_train, y_train)
  yhat = clf.predict(x_test)
  x_true.append(x_test[0])
  y_true.append(y_test[0])
  y_pred_esm.append(yhat[0])
print("training score {:.2f}",format(clf.score(x_train, y_train)))
print("testinging score {:.2f}",format(clf.score(x_true, y_true)))
print(mean_absolute_error(y_true, y_pred_esm))

LeaveOneOut()
training score {:.2f} 0.214758647906799
testinging score {:.2f} 0.2078754145306666
6.533569761426478
